In [1]:
##### BOM_insert_id = ObjectId('575db02205f6a320145f47d0')

import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np

from joblib import Parallel, delayed  
import multiprocessing
num_cores = multiprocessing.cpu_count()

import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.dc_marvel
db.collection_names(include_system_collections=False)




[]

In [2]:
# db.box_office_mojo.drop()
# db.omdb_critics.drop()
# db.collection_names(include_system_collections=False)

In [3]:
mongo = db.box_office_mojo

In [10]:
Marvel_titles  = ["Howard The Duck", "Blade",
           "X-Men","Blade II",
           "Spider-Man","Daredevil",
           "X2: X-Men United","Hulk",
           "The Punisher","Spider-Man 2",
           "Blade: Trinity", "Elektra",
           "Fantastic Four","X-Men: The Last Stand",
           "Ghost Rider","Spider-Man 3",
           "Fantastic Four(2005): Rise of the Silver Surfer","Iron Man",
           "The Incredible Hulk","Punisher: War Zone",
           "X-Men Origins: Wolverine","Iron Man 2",
           "Thor","X-Men: First Class",
           "Captain America: The First Avenger",
           "Ghost Rider: Spirit of Vengeance",
           "Marvel's The Avengers","The Amazing Spider-Man",
           "Iron Man 3","The Wolverine",
           "Thor: The Dark World","Captain America: Winter Soldier",
           "The Amazing Spider-Man 2","X-Men: Days of Future Past",
           "Guardians of the Galaxy","Big Hero 6",
           "Avengers: Age of Ultron", "Ant-Man",
           "Fantastic Four(2015)", "Deadpool",
           "Captain America: Civil War"]


Marvel =["howardtheduck", "blade", 
         "xmen","blade2",
         "spiderman","daredevil",
         "x2","hulk",
         "punisher","spiderman2",
         "blade3", "elektra",
                   "fantasticfour","x3",
                   "ghostrider","spiderman3",
                   "fantasticfour2","ironman",
                   "incrediblehulk","punisher2",
                   "wolverine","ironman2",
                   "thor","xmenfirstclass",
                   "captainamerica","ghostrider2",
                   "avengers11","spiderman4",
                   "ironman3","wolverine2",
                   "thor2","marvel14b",
                   "amazingspiderman2","xmen2014",
                   "marvel2014a","disney2014",
                   "avengers2", "antman",
                   "fantasticfour15", "deadpool2016",
                   "marvel2016"]


Dc_titles = ["Superman","Superman II",
      "Superman III", "Supergirl",
      "Superman IV: The Quest for Peace","Batman",
      "Batman Returns","Batman: Mask of the Phantasm",
      "Batman Forever", "Batman Robin", 
      "Steel", "Catwoman", 
      "Constantine", "Batman Begins",
      "V for Vendetta", "Superman Returns", 
      "The Dark Knight", "Watchmen",
      "The Losers","Jonah Hex","Red",
      "Green Lantern", "The Dark Knight Rises",
      "Man of Steel","Red 2",
      "Batman v Superman: Dawn of Justice"]



Dc = ["superman","superman2",
      "superman3", "supergirl",
      "superman4","batman",
      "batmanreturns","batmanmaskofthephantasm",
      "batmanforever", "batmanrobin", 
      "steel", "catwoman", 
      "constantine", "batmanbegins",
      "vforvendetta", "superman06", 
      "darkknight", "watchmen",
      "losers","jonahhex","red2010",
      "greenlantern", "batman3",
      "superman2012","red2",
      "superman2015"]

In [11]:
def getMovieData(name):
    
    import json
    import boxofficemojoAPI as bom
   
    
    bo = bom.BoxOfficeMojo()
    bo.crawl_for_urls()
    data1 = bo.get_movie_summary(name)
    data1.clean_data()
    data1 = json.loads(data1.to_json())
    
    data2 = bo.get_weekly_summary(name)
    data2.clean_data()
    data2 = json.loads(data2.to_json())['weekly']
    
    Features = dict()
    try:
        Features['director'] = data1["directors"]
    except KeyError:
        Features['director'] = None
        
    try:
        Features['distributor'] = data1["distributor"]
    except KeyError:
        Features['distributor'] = None
        
    try:
        Features['domestic_BO'] = data1["domestic"]
    except KeyError:
        Features['domestic_BO'] = None
    try:
        Features['foreign_BO'] = data1["foreign"]
    except KeyError:
        Features['foreign_BO'] = None
    
    try:
        Features['genre'] = data1["genre"]
    except KeyError:
        Features['genre'] = None
        
    try:
        Features['rating'] = data1["mpaa_rating"]
    except KeyError:
         Features['rating'] = None
    
#     try:
#         Features['release'] = data1["release_date"]
#     except KeyError:
#         Features['release'] = None
        
    try:
        Features['runtime'] = data1["runtime"]
    except KeyError:
        Features['runtime'] = None
        
    try:
        Features['writers'] = data1["writers"]
    except KeyError:
        Features['writers'] = None
        
    
    starting_week = data2[0]['week_number']
    if starting_week == 0:
        del data2[0]
        
    del starting_week

    for k in range(min(len(data2), 15)):
        if k == 0:
            
            try:
                Features['_'.join(('week', str(k+1),'gross'))] = data2[k]['gross']
            except KeyError:
                Features['_'.join(('week', str(k+1),'gross'))] = None
                
            try:
                Features['_'.join(('week', str(k+1),'rank'))] = data2[k]['rank']
            except KeyError:
                Features['_'.join(('week', str(k+1),'rank'))] = None
                
            try:
                Features['_'.join(('week', str(k+1),'avg'))] = data2[k]['average_per_theatre']
            except KeyError:
                Features['_'.join(('week', str(k+1),'avg'))] = None
                
        else :

            Features['_'.join(('week', str(k+1), 'change'))]= data2[k]['week_over_week_change']
            Features['_'.join(('week', str(k+1), 'rank'))]= data2[k]['rank']
            Features['_'.join(('week', str(k+1),'avg'))] = data2[k]['average_per_theatre']
            
    return Features
            
     

        
# def getAllData(Marvel, Dc):
    

#     MARVEL = dict()
#     DC = dict()
#     Result = dict()

#     for name in Marvel:
#         MARVEL[name] = getMovieData(name)

#     for name in Dc:
#         DC[name] = getMovieData(name)

#     MARVEL = json.dumps(MARVEL,  sort_keys=True, separators=(',', ':'), indent =4)
#     DC = json.dumps(DC,  sort_keys=True, separators=(',', ':'), indent =4)
#     Result['marvel'] = MARVEL
#     Result['dc'] = DC
#     Result = json.dumps(Result,  sort_keys=True, separators=(',', ':'), indent =4)
    
#     return Result




In [13]:
%%capture


# DC = Parallel(n_jobs=num_cores)(delayed(getMovieData)(name) for name in Dc)
# MARVEL = Parallel(n_jobs=num_cores)(delayed(getMovieData)(name) for name in Marvel)

M = {}
D = {}
for k in range(len(MARVEL)):
    M[Marvel_titles[k]]= MARVEL[k]
    
for k in range(len(DC)):
    D[Dc_titles[k]]= DC[k]

    
Data = {}
Data['Marvel'] = M
Data['DC'] = D

del M
del D
del Marvel_titles
del Dc_titles



BOM_insert_id = mongo.insert_one(Data).inserted_id
db.collection_names(include_system_collections=False)


[u'box_office_mojo']

In [14]:

client.close
del Data

In [15]:
BOM_insert_id 

ObjectId('575db02205f6a320145f47d0')